In [1]:
from selenium import webdriver
import pandas as pd
from sqlalchemy import create_engine
from selenium.webdriver.chrome.options import Options


#Variables

In [2]:
page=1
url = 'https://zyntern.com/jobs?page='+str(page)
hirdetesLeiras = {
    'Indicator' : [],
    'info': [],
    'details' : [],
    'bottom-location': [],
    'link' : []
 }
databasename = str(pd.to_datetime("today").date()) +' zynter_com.db'
engine = create_engine('sqlite:///' + databasename, echo=False)


In [3]:

chrome_options = Options()
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--headless")
chrome_options.add_argument("--log-level=3")
driver = webdriver.Chrome(options=chrome_options) #options=chrome_options
driver.get(url)

# A szükséges információk összegyüjtése.

In [4]:
allasokSzama = driver.find_elements_by_tag_name('h1')[0].text #állások száma

In [5]:
def dictappend(hirdetesekKontener):
    for i in range(len(hirdetesekKontener)):
        indexincrement = len(hirdetesLeiras['Indicator'])
        hirdetesLeiras['Indicator'].append(indexincrement)
        hirdetesLeiras['link'].append(hirdetesekKontener[i].get_attribute('href'))
        job = hirdetesekKontener[i].find_elements_by_xpath('./div')
        for j in range(len(job)):
            if job[j].get_attribute('class') == 'info':
                    hirdetesLeiras[ 'info'].append(job[j].text.replace("\n" , " "))
            elif   job[j].get_attribute('class') == 'details'  :
                    hirdetesLeiras[ 'details'].append(job[j].text.replace("\n" , " "))
            elif  job[j].get_attribute('class') =='bottom-location':
                hirdetesLeiras[ 'bottom-location'].append(job[j].text)

In [6]:
nincstobboldal = driver.find_elements_by_xpath("//*[contains(text() , 'Nincs találat')]" )


while len(nincstobboldal) == 0 : 
    url2 = 'https://zyntern.com/jobs?page='+str(page)
    driver.get(url2)
    driver.implicitly_wait(10)
    hirdetesekKontener = driver.find_elements_by_css_selector('#job_listing_container > div > div > div.listings>a')
    dictappend(hirdetesekKontener)
    page=page+1
    nincstobboldal = driver.find_elements_by_xpath("//*[contains(text() , 'Nincs találat')]" )
    print(url2)
    print(f"Eddig kinyert adat : {len(hirdetesLeiras['Indicator'])} ")
print(f"Az oldalon talált állások száma : {allasokSzama} ebből a mennyiségből kinyert addat : {len(hirdetesLeiras['Indicator'])} ")  

https://zyntern.com/jobs?page=1
Eddig kinyert adat : 12 
https://zyntern.com/jobs?page=2
Eddig kinyert adat : 24 
https://zyntern.com/jobs?page=3
Eddig kinyert adat : 36 
https://zyntern.com/jobs?page=4
Eddig kinyert adat : 48 
https://zyntern.com/jobs?page=5
Eddig kinyert adat : 60 
https://zyntern.com/jobs?page=6
Eddig kinyert adat : 72 
https://zyntern.com/jobs?page=7
Eddig kinyert adat : 84 
https://zyntern.com/jobs?page=8
Eddig kinyert adat : 96 
https://zyntern.com/jobs?page=9
Eddig kinyert adat : 108 
https://zyntern.com/jobs?page=10
Eddig kinyert adat : 120 
https://zyntern.com/jobs?page=11
Eddig kinyert adat : 132 
https://zyntern.com/jobs?page=12
Eddig kinyert adat : 144 
https://zyntern.com/jobs?page=13
Eddig kinyert adat : 156 
https://zyntern.com/jobs?page=14
Eddig kinyert adat : 168 
https://zyntern.com/jobs?page=15
Eddig kinyert adat : 180 
https://zyntern.com/jobs?page=16
Eddig kinyert adat : 192 
https://zyntern.com/jobs?page=17
Eddig kinyert adat : 204 
https://zynter

In [7]:
df = pd.DataFrame(hirdetesLeiras)
df = df.drop('Indicator', axis=1)
df.to_excel(str(pd.to_datetime("today").date()) + ' zyntern_com.xlsx')
df.to_sql('hirdetesLeiras', con=engine, if_exists='append')
driver.quit()